In [1]:
from src.Webscraping import Webscraping_TrailData as TD
from src.Webscraping import Webscraping_TrailReports as TR
import requests
from bs4 import BeautifulSoup
from pymongo import MongoClient
import pymongo
import math
import pandas as pd
from multiprocessing.dummy import Pool
from multiprocessing import cpu_count
import numpy as np 

In [2]:
all_hike_urls = pd.read_csv('data/WTA_all_trail_data_clean.csv')

In [42]:
all_hike_urls

,Unnamed: 0,Coast,Dogs not allowed,Established campsites,Fall foilage,Good for kids,Lakes,Mountain views,Old growth,Ridges/passes,Rivers,Summits,Waterfalls,Wildflowers/Meadows,Wildlife,elevation_gain,highest_point,hike_name,lat,long,numReports,number_votes,url,which_pass,super_region,sub_region,total_distance,stars
0,0,0,0,0,0,0,0,1,0,0,0,0,1,1,0,3800.0,4788.0,Mount Teneriffe,47.4861,-121.7011,451,18.0,https://www.wta.org/go-hiking/hikes/mount-tene...,Discover Pass,Snoqualmie Region,North Bend Area,13.8,3.44
1,1,0,0,0,0,0,0,1,0,0,0,0,1,1,0,1585.0,2370.0,Teneriffe Falls (Kamikaze Falls),47.4862,-121.6995,683,35.0,https://www.wta.org/go-hiking/hikes/kamikaze-f...,Discover Pass,Snoqualmie Region,North Bend Area,5.4,3.89
2,2,0,0,0,0,0,0,0,0,0,1,0,0,1,1,NaN,NaN,Chief Joseph Wildlife Area - 4-O Ranch,46.0407,-117.2912,1,NaN,https://www.wta.org/go-hiking/hikes/chief-jose...,Discover Pass,NaN,NaN,NaN,NaN
3,3,0,0,0,0,0,0,1,0,1,0,1,0,1,0,3020.0,5220.0,West Defiance - Putrid Pete's Peak,47.4267,-121.5864,114,7.0,https://www.wta.org/go-hiking/hikes/west-defiance,Northwest Forest Pass,Snoqualmie Region,North Bend Area,6.8,3.86
4,4,0,0,0,0,0,1,1,0,0,0,0,0,0,0,1400.0,1950.0,Diablo Lake,48.7206,-121.1216,101,15.0,https://www.wta.org/go-hiking/hikes/diablo-lake,NaN,North Cascades,North Cascades Highway - Hwy 20,7.6,3.20
5,5,0,0,0,0,1,0,0,0,0,0,0,0,0,0,110.0,1270.0,Klondike Swamp,47.5335,-122.1137,42,4.0,https://www.wta.org/go-hiking/hikes/klondike-s...,None,Issaquah Alps,Cougar Mountain,2.5,2.75
6,6,0,0,0,0,1,0,1,0,1,0,1,0,1,0,1685.0,8245.0,Tiffany Mountain,48.6630,-119.9648,46,10.0,https://www.wta.org/go-hiking/hikes/tiffany-mo...,NaN,North Cascades,Pasayten,4.2,4.10
7,7,0,0,0,0,1,0,0,0,0,0,0,0,1,1,900.0,2900.0,Colville Mountain Loop Trail,NaN,NaN,1,0.0,https://www.wta.org/go-hiking/hikes/colville-m...,None,Eastern Washington,Selkirk Range,4.0,0.00
8,8,0,0,0,0,1,0,0,0,0,0,0,0,1,1,NaN,NaN,Scatter Creek,46.8432,-122.9842,0,NaN,https://www.wta.org/go-hiking/hikes/scatter-creek,NaN,NaN,NaN,NaN,NaN
9,9,0,0,0,0,1,0,0,1,0,1,0,1,1,1,40.0,1360.0,Camp Creek Falls,46.4487,-121.8355,3,4.0,https://www.wta.org/go-hiking/hikes/camp-creek...,Northwest Forest Pass,South Cascades,White Pass/Cowlitz River Valley,0.6,3.50


In [6]:
all_hike_urls['numReports'].min()

0

In [40]:
pd.set_option('display.max_columns', None)
%matplotlib inline

In [52]:
blanks = all_hike_urls[all_hike_urls['lat'].isna()]
blanks

,Unnamed: 0,Coast,Dogs not allowed,Established campsites,Fall foilage,Good for kids,Lakes,Mountain views,Old growth,Ridges/passes,Rivers,Summits,Waterfalls,Wildflowers/Meadows,Wildlife,elevation_gain,highest_point,hike_name,lat,long,numReports,number_votes,url,which_pass,super_region,sub_region,total_distance,stars
7,7,0,0,0,0,1,0,0,0,0,0,0,0,1,1,900.0,2900.0,Colville Mountain Loop Trail,NaN,NaN,1,0.0,https://www.wta.org/go-hiking/hikes/colville-m...,None,Eastern Washington,Selkirk Range,4.00,0.00
12,12,0,0,1,0,0,1,1,1,1,1,0,0,0,1,3600.0,4900.0,Stehekin to Rainy Pass,NaN,NaN,1,0.0,https://www.wta.org/go-hiking/hikes/stehekin-t...,National Park Pass,North Cascades,North Cascades Highway - Hwy 20,37.40,0.00
16,16,0,0,0,0,0,1,1,1,0,0,0,0,0,0,8600.0,6400.0,Chinook Pass to Snoqualmie Pass,NaN,NaN,2,0.0,https://www.wta.org/go-hiking/hikes/chinook-pa...,Wilderness permit. Self-issue at trailhead (no...,Mount Rainier Area,Chinook Pass - Hwy 410,126.60,0.00
18,18,0,0,1,0,0,1,1,0,1,1,0,1,1,0,5200.0,6700.0,Sunrise to Carbon River Traverse,NaN,NaN,1,0.0,https://www.wta.org/go-hiking/hikes/sunrise-to...,National Park Pass,Mount Rainier Area,NE - Sunrise/White River,27.40,0.00
23,23,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2000.0,3116.0,Sauer's Mountain,NaN,NaN,101,11.0,https://www.wta.org/go-hiking/hikes/sauer-moun...,None,Central Cascades,Stevens Pass - East,6.00,4.00
91,91,0,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN,NaN,Pilchuck Falls,NaN,NaN,1,NaN,https://www.wta.org/go-hiking/hikes/pilchuck-f...,NaN,NaN,NaN,NaN,NaN
93,93,0,0,0,0,0,0,0,0,0,0,1,0,0,0,NaN,NaN,Red Mountain via Money Creek Road,NaN,NaN,2,NaN,https://www.wta.org/go-hiking/hikes/red-moutai...,None,NaN,NaN,NaN,NaN
121,121,0,0,0,0,0,0,0,1,0,0,0,0,1,1,1750.0,5000.0,Mount Spokane State Park - Trail 110,NaN,NaN,15,4.0,https://www.wta.org/go-hiking/hikes/mount-spok...,Discover Pass,Eastern Washington,Spokane Area/Coeur d'Alene,7.50,4.00
122,122,0,0,0,0,0,0,0,0,0,0,1,0,0,0,NaN,NaN,Mount Howard,NaN,NaN,10,NaN,https://www.wta.org/go-hiking/hikes/mount-howard,NaN,NaN,NaN,NaN,NaN
128,128,0,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN,NaN,Big Chief Mountain,NaN,NaN,5,NaN,https://www.wta.org/go-hiking/hikes/big-chief-...,NaN,NaN,NaN,NaN,NaN


In [88]:
clean_hikes = all_hike_urls.dropna(subset = ['lat','long','elevation_gain','highest_point'])

In [99]:
werk = clean_hikes.sort_values('numReports').drop_duplicates(subset='hike_name', keep= 'last')
olympics = werk[werk['super_region'] == 'Olympic Peninsula']

In [101]:
olympics['sub_region'].unique()

array([' Kitsap Peninsula', ' Hood Canal', ' Northern Coast',
       ' Pacific Coast', ' Olympia', nan], dtype=object)

In [105]:
def get_medians(df):
    medians = {}
    sub_regions = df['sub_region'].unique()
    for region in sub_regions:
        lat = df[df['sub_region'] == region]['lat'].mean()
        lon = df[df['sub_region'] == region]['lat'].mean()
        medians[region] = (lat,lon)
    return medians
        

In [123]:
medians = get_medians(olympics)
ans = medians[' Kitsap Peninsula']

In [125]:
ans[0].type

AttributeError: 'float' object has no attribute 'type'

In [111]:
olympics[olympics['sub_region'] == ' Kitsap Peninsula']['hike_name'].unique()

array(['Clear Creek Trail', 'Banner Forest', 'Anderson Point',
       'Foulweather Bluff Preserve Trail', 'Kopachuck State Park Trail',
       'Illahee Forest Preserve', 'McCormick Forest Park',
       'Harstine Island State Park', 'Penrose Point State Park',
       'Hansville Greenway', 'Mary E. Theler Wetlands Nature Preserve',
       'Newberry Hill Heritage Park', 'Gold Mountain',
       'Green Mountain - Wildcat Trail', 'Guillemot Cove',
       'Green Mountain - Gold Creek Trail'], dtype=object)

In [116]:
oly = olympics.sort_values('sub_region')
oly

,Unnamed: 0,Coast,Dogs not allowed,Established campsites,Fall foilage,Good for kids,Lakes,Mountain views,Old growth,Ridges/passes,Rivers,Summits,Waterfalls,Wildflowers/Meadows,Wildlife,elevation_gain,highest_point,hike_name,lat,long,numReports,number_votes,url,which_pass,super_region,sub_region,total_distance,stars
250,250,0,0,0,0,0,0,1,1,1,0,1,0,1,1,3300.0,5944.0,Mount Ellinor,47.5066,-123.2321,664,78.0,https://www.wta.org/go-hiking/hikes/mount-ellinor,Northwest Forest Pass,Olympic Peninsula,Hood Canal,6.2,4.49
276,276,0,0,0,0,0,0,0,0,1,1,0,0,0,0,2700.0,2500.0,Notch Pass,47.8262,-122.9389,72,8.0,https://www.wta.org/go-hiking/hikes/notch-pass,None,Olympic Peninsula,Hood Canal,8.6,4.00
812,812,1,0,0,0,1,0,0,1,0,1,0,0,0,1,400.0,425.0,Dosewallips State Park - Steam Donkey Trail,47.6876,-122.8999,71,9.0,https://www.wta.org/go-hiking/hikes/dosewallip...,Discover Pass,Olympic Peninsula,Hood Canal,3.5,3.00
1106,1106,0,0,1,0,0,0,0,1,0,1,0,0,0,0,1600.0,4600.0,Tubal Cain Mine,47.9620,-123.1124,71,6.0,https://www.wta.org/go-hiking/hikes/tull-canyon,None,Olympic Peninsula,Hood Canal,7.2,3.00
922,922,0,0,0,0,0,0,1,0,0,0,1,0,1,1,988.0,6988.0,Buckhorn Mountain,47.8268,-123.1166,71,4.0,https://www.wta.org/go-hiking/hikes/buckhorn-m...,Northwest Forest Pass,Olympic Peninsula,Hood Canal,13.5,4.75
1118,1118,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2800.0,5350.0,Deer Ridge,47.9652,-123.1936,60,10.0,https://www.wta.org/go-hiking/hikes/deer-ridge,NaN,Olympic Peninsula,Hood Canal,9.8,2.40
1386,1386,0,0,0,0,1,0,0,0,0,0,0,0,0,0,800.0,1600.0,Dry Creek,47.5015,-123.3206,60,5.0,https://www.wta.org/go-hiking/hikes/dry-creek,Northwest Forest Pass,Olympic Peninsula,Hood Canal,6.0,2.60
762,762,0,0,0,0,0,0,0,0,0,0,0,0,0,0,4300.0,4800.0,Lake Constance,47.7296,-123.1417,59,14.0,https://www.wta.org/go-hiking/hikes/lake-const...,NaN,Olympic Peninsula,Hood Canal,12.6,3.50
3015,3015,0,0,0,0,1,1,0,1,0,0,0,0,0,1,160.0,480.0,Gibbs Lake,47.9704,-122.8108,12,4.0,https://www.wta.org/go-hiking/hikes/gibbs-lake,NaN,Olympic Peninsula,Hood Canal,2.5,3.75
685,685,0,1,1,0,0,0,1,1,0,1,0,1,1,0,2400.0,3600.0,West Fork Dosewallips River,47.7434,-123.2033,57,6.0,https://www.wta.org/go-hiking/hikes/west-fork-...,None,Olympic Peninsula,Hood Canal,15.8,4.33


In [126]:
def region_to_subregion(db):
    split = db['region'].apply(lambda x: str(x).split('--'))
    super_region = []
    sub_region = []
    for trail in split:
        if len(trail) == 1:
            super_region.append(trail[0])
            sub_region.append('NaN')
        else:
            super_region.append(trail[0])
            sub_region.append(trail[1])
    db['super_region'] = super_region
    db['sub_region'] = sub_region
   

In [12]:
def distance(origin, destination):
    lat1, lon1 = origin
    lat2, lon2 = destination
    radius = 6371 # km

    dlat = math.radians(lat2-lat1)
    dlon = math.radians(lon2-lon1)
    a = math.sin(dlat/2) * math.sin(dlat/2) + math.cos(math.radians(lat1)) \
        * math.cos(math.radians(lat2)) * math.sin(dlon/2) * math.sin(dlon/2)
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1-a))
    d = radius * c

    return d

In [30]:
# def distance_from_median(df):
#     ''' Calculated distance from median point. Median is just for hood canal'''
#     distances = []
#     medians = get_medians(df)
#     for region,median in medians:
#         destination = median
#         all_hikes = list(df[df['sub_region'] == region]['hike_name'])
#         for hike in all_hikes:
#             lat1 = df.loc[df['hike_name']== hike]['lat']
#             lon1 = df.loc[df['hike_name']== hike]['long']
#             origin = float(lat1),float(lon1)
#             distances.append(distance(origin,destination))
#     return distances

In [120]:
def distance_from_median(df):
    ''' Calculated distance from median point. Median is just for hood canal'''
    distances = []
    medians = get_medians(df)
    all_hikes = list(df['hike_name'])
    for hike in all_hikes:
        lat1 = df.loc[df['hike_name']== hike]['lat']
        lon1 = df.loc[df['hike_name']== hike]['long']
        origin = float(lat1),float(lon1)
        sub_region = df.loc[df['hike_name']== hike]['sub_region']
        destination = medians[sub_region]
        distances.append(distance(origin,destination))
    return distances

In [121]:
distance_from_median(olympics)

TypeError: 'Series' objects are mutable, thus they cannot be hashed

In [23]:
hike_urls['super_region'] = hike_urls[~hike_urls['region'].isna()]['region'].apply(lambda x: str(x).split('--')[0])
hike_urls['sub_region'] = hike_urls[~hike_urls['region'].isna()]['region'].apply(lambda x: str(x).split('--')[1])

In [33]:
hike_urls['total_distance'] = hike_urls[~hike_urls['distance'].isna()]['distance'].apply(lambda x: total_dst(x))

In [29]:
hike_urls['stars'] = hike_urls[~hike_urls['rating'].isna()]['rating'].apply(lambda x: stars(x))

In [45]:
hike_urls['number_votes'] = hike_urls['number_votes'].apply(lambda x: float(x))

In [28]:

def stars(string):
    '''Extracts number of stars'''
    lst = string.split()
    return float(lst[0])

In [31]:
def total_dst(string):
    """Extracts mileage from string and calculates total mileage depending on if
    description says one-way or roundtrip."""
    miles = []
    try:
        for s in string.split():
            miles.append(float(s))
    except:
        miles.append(string)
    if 'roundtrip' in string:
        return miles[0]
    elif 'one-way' in string:
        return miles[0]*2
    else:
        return miles[0]

In [ ]:
def stars(string):
    '''Extracts number of stars'''
    lst = string.split()
    return float(lst[0])

def total_dst(string):
    """Extracts mileage from string and calculates total mileage depending on if
    description says one-way or roundtrip."""
    miles = []
    try:
        for s in string.split():
            miles.append(float(s))
    except:
        miles.append(string)
    if 'roundtrip' in string:
        return miles[0]
    elif 'one-way' in string:
        return miles[0]*2
    else:
        return miles[0]

hike_urls = pd.read_csv('Olympic_trails_clean.csv')
hike_urls['super_region'] = hike_urls[~hike_urls['region'].isna()]['region'].apply(lambda x: str(x).split('--')[0])
hike_urls['sub_region'] = hike_urls[~hike_urls['region'].isna()]['region'].apply(lambda x: str(x).split('--')[1])
hike_urls['total_distance'] = hike_urls[~hike_urls['distance'].isna()]['distance'].apply(lambda x: total_dst(x))
hike_urls['stars'] = hike_urls[~hike_urls['rating'].isna()]['rating'].apply(lambda x: stars(x))
hike_urls['number_votes'] = hike_urls['number_votes'].apply(lambda x: float(x))

AttributeError: 'list' object has no attribute 'join'

In [179]:
    mc = pymongo.MongoClient()
    db = mc['wta']
    trail_reports = db['trail_reports']
    raw_html = db['html']
    trail_page_raw_html = db['trail_html']

In [180]:
df = pd.DataFrame(list(trail_reports.find()))

In [181]:
df

,Creator,Date,Report,Trail,Trail_condtions,Votes,_id
0,\nrustysailor,"May 06, 2018",Arrived at the trailhead at about 11am to find...,Mount Rose,"\nBeware of:\n snow, trail condit...",3,5af25573acf3d692490fdf8e
1,\nkilpatjw,"May 05, 2018",Got to the trailhead at around 9am and the lot...,Mount Rose,\nBeware of:\n trail conditions\n...,3,5af25573acf3d692490fdf8f
2,\njalaugh,"May 05, 2018",Shortly after the very well marked summit loop...,Mount Rose,"\nBeware of:\n snow, trail condit...",6,5af25573acf3d692490fdf90
3,\nBecuz Its There,"Apr 25, 2018",Made another mad dash up Rose for training. T...,Mount Rose,"\nBeware of:\n snow, trail condit...",7,5af25573acf3d692490fdf91
4,\nCloud Chaser,"Apr 22, 2018","My wife, dog & I headed up Mt Rose this weeken...",Mount Rose,\nBeware of:\n trail conditions\n...,1,5af25573acf3d692490fdf92
5,\nStepbyStep,"Apr 21, 2018",Lots of snow on the loop. Micro spike and pol...,Mount Rose,\nBeware of:\n snow conditions\n ...,4,5af25573acf3d692490fdf94
6,\ntpioletti,"Apr 21, 2018",Parking lot is pretty small but more space on ...,Mount Rose,None,5,5af25573acf3d692490fdf95
7,\nIrene Grinberg,"Apr 20, 2018","Road has some potholes on the way, definitely ...",Mount Rose,"\nBeware of:\n road, snow & trail...",1,5af25573acf3d692490fdf96
8,\nIrene Grinberg,"Apr 20, 2018","Road has some potholes on the way, definitely ...",Mount Rose,"\nBeware of:\n road, snow & trail...",None,5af25573acf3d692490fdf97
9,\nNicole A,"Apr 19, 2018",Started off this sunny morning heading to the ...,Mount Rose,"\nBeware of:\n snow, trail condit...",5,5af25573acf3d692490fdf98


In [77]:
df['Date_type'] = pd.to_datetime(df['Date'])

In [ ]:
df['Creator'] = df['Creator'].apply(lambda x: x.strip('\n'))

In [183]:
len(df['Trail'].unique())

29

In [184]:
db.trail_reports.count()

57732